In [69]:
from math import log2


class ID3:
    def __init__(self):
        self.mins_steps = {}
        self.tree = {}
        self.majority_class = 'hi_classssss'
        
        
    # return records if feature_name == value 
    def data_filter(self, data, feature_name, value):
        new_data = {}
        for key in list(data.keys()):
            new_data[key] = []

        for i in range(len(data[feature_name])):
            if data[feature_name][i] == value:
                for key in data.keys():
                    new_data[key].append(data[key][i])

        return new_data



    # Find the entropy of the given feature
    def entropy(self, feature_column):
        # find the frequency of each unique value for the given feature
        counts = [feature_column.count(i) for i in list(set(feature_column))]

        total_entropy = 0

        for count in counts:
            probability = count/sum(counts)
            #total_entropy += (-probability * log2(probability))
            total_entropy -= probability * log2(probability)

        return total_entropy


    # Informatino Gain = Entropy(parent) - [average entropy(children)]  
    def information_gain(self, data, feature_name, label_name):
        # find the entropy of given data
        parent_entropy = self.entropy(data[label_name])

        # find unique values and their frequency for the attribute to be split
        freqs = dict( (i, data[feature_name].count(i)) for i in 
                 list(set( data[feature_name] )) )

        # calculate weighted entropy of all subsets
        total_weighted_entropy = 0

        for v, freq in freqs.items():
            subset_probability = freq/sum(freqs.values())

            subset_entropy = self.entropy(self.data_filter(data, feature_name, v)[label_name])
            total_weighted_entropy += (subset_probability*subset_entropy)


        # calculate information gain
        information_gain = parent_entropy - total_weighted_entropy

        return information_gain


    # creating the decision tree
    def train(self, data, label_name, discrete_attributes):
        feature_names = list(data.keys())
        feature_names.remove(label_name)

        # determine majority class
        class_freqs = dict( (i, data[label_name].count(i)) for i in \
                             list(set( data[label_name] )) )
        self.majority_class = sorted(class_freqs, key=lambda x : class_freqs[x], reverse=True)[0]
        
        for attr in discrete_attributes:
            data[attr], self.mins_steps[attr] = self.discretize_list(data[attr])
        
        self.tree = self.build_tree(data, data, feature_names, label_name, \
                                    self.majority_class, self.majority_class)
        
        
    # creating the decision tree
    def build_tree(self, data, original_data, feature_names, label_name, \
                      majority_class, parent_node_class=None):
        # if data set contains no features to train with, return parent node class
        if len(feature_names) == 0:
            #print("1:", parent_node_class)
            return parent_node_class
        # if subset is empty, ie. no samples, return majority class of original data
        elif len(data[label_name]) == 0:
            #print("2:", majority_class)
            return majority_class
        # if data is pure, return the majority class of subset
        elif len(list(set( data[label_name] ))) <= 1:
            #print("3:", '3'+list(set( data[label_name] ))[0])
            return list(set( data[label_name] ))[0]
        # if none of the above are true, construct a branch:
        else:
            # determine parent node class of current branch
            class_freqs = dict( (i, data[label_name].count(i)) for i in \
                         list(set( data[label_name] )) )
            parent_node_class = sorted(class_freqs, key=lambda x : class_freqs[x], reverse=True)[0]

            # determine information gain values for each feature
            # choose feature which best splits the data, ie. highest value
            gains = dict( (feature, self.information_gain(data, feature, label_name)) for \
                               feature in feature_names ) 
            best_feature = sorted(gains, key=lambda x : gains[x], reverse=True)[0]


            # create tree structure, empty at first
            tree = {best_feature: {}}

            # remove best feature from available features, it will become the parent node
            #feature_names.remove(best_feature)
            feature_names = [i for i in feature_names if i != best_feature]


            # create nodes under parent node
            for value in list(set( data[best_feature] )): #or original_data[best_feature]
                new_branch_data = self.data_filter(data, best_feature, value)

                # call the algorithm recursively
                new_branch = self.build_tree(new_branch_data, original_data, feature_names, \
                                           label_name, majority_class, parent_node_class)

                # add subtree to original tree
                tree[best_feature][value] = new_branch

            return tree
    
    
    # making prediction for an unseen instance using created tree
    def classify(self, instance, current_tree):
        # map instance data to tree
        for attribute in list(instance.keys()):
            # check if feature exists in tree
            if attribute in list(current_tree.keys()):
                try:
                    result = current_tree[attribute][instance[attribute]]
                except:
                    return self.majority_class

                result = current_tree[attribute][instance[attribute]]

                # if more attributes exist within result, recursively find best result
                if isinstance(result, dict):
                    return self.classify(instance, result)
                else:
                    return result
            

    # take a list of instances which each one is a dictionary
    def test(self, test_data, discrete_attributes):
        predictions = []

        # make a prediction for every instance
        feature_names = list(test_data.keys())
        length = len(test_data[ feature_names[0] ])
        for i in range(length):
            test_instance = { feature:test_data[feature][i] for feature in feature_names }

            for attr in discrete_attributes:
                test_instance[attr] = self.discretize_value(self.mins_steps[attr][0], \
                                                            self.mins_steps[attr][1], \
                                                            test_instance[attr])
            predictions.append(self.classify(test_instance, self.tree))

        return predictions


    def discretize_list(self, l):
        min_value = min(l)
        step = ( max(l) - min(l) ) / 10
        ranges = []
        for i in range(10):
            ranges.append(min_value + (i * step))

        new_l = []
        for value in l:
            new_value = 10
            for index_range in zip(range(len(ranges)), ranges):
                if value < index_range[1]:
                    new_value = index_range[0]
                    break
            new_l.append(new_value)
        return new_l, (min_value, step)

    def discretize_value(self, min_value, step, value):
        ranges = []
        for i in range(10):
            ranges.append(min_value + (i * step))

        new_value = 10
        for index_range in zip(range(len(ranges)), ranges):
            if value < index_range[1]:
                new_value = index_range[0]
                return new_value
        return new_value

In [32]:
# I use the below libraries only to read the data from files.
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# breast-cancer-wisconsin dataset

## One Run

In [43]:
df = pd.read_csv('./breast-cancer-wisconsin.data', header=None)
# drop rows with missing values, missing = ?
df = df.replace("?", np.nan)
df = df.dropna()
# organize data into input and output
X = df.drop(columns=10)
y = df[10]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
d_demo = pd.concat([X_train, y_train], axis=1).to_dict(orient='list')
d_test = X_test.to_dict(orient="list")

feature_names = list(d_demo.keys())
label_name = feature_names[-1]
discrete_attributes = []


classifier = ID3()
classifier.train(d_demo, label_name, discrete_attributes)
my_pred = classifier.test(d_test, discrete_attributes)
accuracy_score(y_test, my_pred)

0.7168141592920354

## 50 runs (10 * 5 folds)

In [44]:
from sklearn.metrics import accuracy_score 
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold

df = pd.read_csv('./breast-cancer-wisconsin.data', header=None)
# drop rows with missing values, missing = ?
df = df.replace("?", np.nan)
df = df.dropna()
df = df.reset_index()

target = df[10]
skf = StratifiedKFold(n_splits=5, shuffle=True)
accs = []
for run in range(10):
    print("================Run {}================".format(run))
    fold_no = 1
    for train_index, test_index in skf.split(df, target):
        train = df.loc[train_index,:]
        test = df.loc[test_index,:]
        
        X = train.drop(columns=10)
        y = train[10]
        d_demo = pd.concat([X, y], axis=1).to_dict(orient='list')
        
        X_test = test.drop(columns=10)
        y_test = test[10]
        d_test = X_test.to_dict(orient="list")

        feature_names = list(d_demo.keys())
        label_name = feature_names[-1]
        discrete_attributes = []
        
        classifier = ID3()
        classifier.train(d_demo, label_name, discrete_attributes)
        my_pred = classifier.test(d_test, discrete_attributes)

        print("fold: ", fold_no, "===>", "accuracy: ", accuracy_score(y_test, my_pred))
        accs.append(accuracy_score(y_test, my_pred))
        fold_no += 1


================Run 0================
fold:  1 ===> accuracy:  0.6496350364963503
fold:  2 ===> accuracy:  0.6496350364963503
fold:  3 ===> accuracy:  0.6496350364963503
fold:  4 ===> accuracy:  0.6544117647058824
fold:  5 ===> accuracy:  0.6470588235294118
================Run 1================
fold:  1 ===> accuracy:  0.6496350364963503
fold:  2 ===> accuracy:  0.6496350364963503
fold:  3 ===> accuracy:  0.6496350364963503
fold:  4 ===> accuracy:  0.6544117647058824
fold:  5 ===> accuracy:  0.6470588235294118
================Run 2================
fold:  1 ===> accuracy:  0.6496350364963503
fold:  2 ===> accuracy:  0.6496350364963503
fold:  3 ===> accuracy:  0.6496350364963503
fold:  4 ===> accuracy:  0.6544117647058824
fold:  5 ===> accuracy:  0.6470588235294118
================Run 3================
fold:  1 ===> accuracy:  0.6496350364963503
fold:  2 ===> accuracy:  0.6496350364963503
fold:  3 ===> accuracy:  0.6496350364963503
fold:  4 ===> accuracy:  0.6544117647058824
fold:  5 ===

In [45]:
len(accs)

50

In [46]:
import statistics
print("Standard deviation: ", statistics.stdev(accs) )
print("Mean: ", statistics.mean(accs) )

Standard deviation:  0.0024110947709128448
Mean:  0.6500751395448691


# car dataset

## One Run

In [52]:
df = pd.read_csv('./car.data', header=None)
# organize data into input and output
X = df.drop(columns=6)
y = df[6]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
d_demo = pd.concat([X_train, y_train], axis=1).to_dict(orient='list')
d_test = X_test.to_dict(orient="list")

feature_names = list(d_demo.keys())
label_name = feature_names[-1]
discrete_attributes = []

classifier = ID3()
classifier.train(d_demo, label_name, discrete_attributes)
my_pred = classifier.test(d_test, discrete_attributes)
accuracy_score(y_test, my_pred)

0.9054290718038529

## 50 runs (10 * 5 folds)

In [53]:
from sklearn.metrics import accuracy_score 
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold

df = pd.read_csv('./car.data', header=None)
target = df[6]
skf = StratifiedKFold(n_splits=5, shuffle=True)
accs = []
for run in range(10):
    print("================Run {}================".format(run))
    fold_no = 1
    for train_index, test_index in skf.split(df, target):
        train = df.loc[train_index,:]
        test = df.loc[test_index,:]
        
        X = train.drop(columns=6)
        y = train[6]
        d_demo = pd.concat([X, y], axis=1).to_dict(orient='list')
        
        X_test = test.drop(columns=6)
        y_test = test[6]
        d_test = X_test.to_dict(orient="list")

        feature_names = list(d_demo.keys())
        label_name = feature_names[-1]
        discrete_attributes = []
        
        classifier = ID3()
        classifier.train(d_demo, label_name, discrete_attributes)
        my_pred = classifier.test(d_test, discrete_attributes)

        print("fold: ", fold_no, "===>", "accuracy: ", accuracy_score(y_test, my_pred))
        accs.append(accuracy_score(y_test, my_pred))
        fold_no += 1


================Run 0================
fold:  1 ===> accuracy:  0.8959537572254336
fold:  2 ===> accuracy:  0.9219653179190751
fold:  3 ===> accuracy:  0.869942196531792
fold:  4 ===> accuracy:  0.8985507246376812
fold:  5 ===> accuracy:  0.8898550724637682
================Run 1================
fold:  1 ===> accuracy:  0.9161849710982659
fold:  2 ===> accuracy:  0.9104046242774566
fold:  3 ===> accuracy:  0.8670520231213873
fold:  4 ===> accuracy:  0.8753623188405797
fold:  5 ===> accuracy:  0.8927536231884058
================Run 2================
fold:  1 ===> accuracy:  0.884393063583815
fold:  2 ===> accuracy:  0.9190751445086706
fold:  3 ===> accuracy:  0.8728323699421965
fold:  4 ===> accuracy:  0.8927536231884058
fold:  5 ===> accuracy:  0.8985507246376812
================Run 3================
fold:  1 ===> accuracy:  0.884393063583815
fold:  2 ===> accuracy:  0.9017341040462428
fold:  3 ===> accuracy:  0.9046242774566474
fold:  4 ===> accuracy:  0.8869565217391304
fold:  5 ===> a

In [54]:
len(accs)

50

In [55]:
import statistics
print("Standard deviation: ", statistics.stdev(accs) )
print("Mean: ", statistics.mean(accs) )

Standard deviation:  0.015420059789235005
Mean:  0.8939798944458407


# mushroom dataset

## One Run

In [58]:
df = pd.read_csv('./mushroom.data', header=None)
# organize data into input and output
X = df.drop(columns=0)
y = df[0]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
d_demo = pd.concat([X_train, y_train], axis=1).to_dict(orient='list')
d_test = X_test.to_dict(orient="list")
del d_demo[11]
del d_demo[16]
del d_test[11]
del d_test[16]

feature_names = list(d_demo.keys())
label_name = feature_names[-1]
discrete_attributes = []

classifier = ID3()
classifier.train(d_demo, label_name, discrete_attributes)
my_pred = classifier.test(d_test, discrete_attributes)
accuracy_score(y_test, my_pred)

1.0

## 50 runs (10 * 5 folds)

In [59]:
from sklearn.metrics import accuracy_score 
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold

df = pd.read_csv('./mushroom.data', header=None)
target = df[0]
skf = KFold(n_splits=5, shuffle=True)
accs = []
for run in range(10):
    print("================Run {}================".format(run))
    fold_no = 1
    for train_index, test_index in skf.split(df, target):
        train = df.loc[train_index,:]
        test = df.loc[test_index,:]
        
        X = train.drop(columns=0)
        y = train[0]
        d_demo = pd.concat([X, y], axis=1).to_dict(orient='list')

        X_test = test.drop(columns=0)
        y_test = test[0]
        d_test = X_test.to_dict(orient="list")
        
        del d_demo[11]
        del d_demo[16]
        del d_test[11]
        del d_test[16]
        
        feature_names = list(d_demo.keys())
        label_name = feature_names[-1]
        discrete_attributes = []

        classifier = ID3()
        classifier.train(d_demo, label_name, discrete_attributes)
        my_pred = classifier.test(d_test, discrete_attributes)

        print("fold: ", fold_no, "===>", "accuracy: ", accuracy_score(y_test, my_pred))
        accs.append(accuracy_score(y_test, my_pred))
        fold_no += 1


================Run 0================
fold:  1 ===> accuracy:  1.0
fold:  2 ===> accuracy:  1.0
fold:  3 ===> accuracy:  1.0
fold:  4 ===> accuracy:  1.0
fold:  5 ===> accuracy:  1.0
================Run 1================
fold:  1 ===> accuracy:  1.0
fold:  2 ===> accuracy:  1.0
fold:  3 ===> accuracy:  1.0
fold:  4 ===> accuracy:  1.0
fold:  5 ===> accuracy:  1.0
================Run 2================
fold:  1 ===> accuracy:  1.0
fold:  2 ===> accuracy:  1.0
fold:  3 ===> accuracy:  1.0
fold:  4 ===> accuracy:  1.0
fold:  5 ===> accuracy:  1.0
================Run 3================
fold:  1 ===> accuracy:  1.0
fold:  2 ===> accuracy:  1.0
fold:  3 ===> accuracy:  1.0
fold:  4 ===> accuracy:  1.0
fold:  5 ===> accuracy:  1.0
================Run 4================
fold:  1 ===> accuracy:  1.0
fold:  2 ===> accuracy:  1.0
fold:  3 ===> accuracy:  1.0
fold:  4 ===> accuracy:  1.0
fold:  5 ===> accuracy:  1.0
================Run 5================
fold:  1 ===> accuracy:  1.0
fold:  2 ===> accu

In [60]:
len(accs)

50

In [61]:
import statistics
print("Standard deviation: ", statistics.stdev(accs) )
print("Mean: ", statistics.mean(accs) )

Standard deviation:  0.0
Mean:  1.0


# ecoli dataset

## One Run

In [70]:
df = pd.read_csv('./ecoli.data', header=None)
# organize data into input and output
X = df.drop(columns=8)
y = df[8]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
d_demo = pd.concat([X_train, y_train], axis=1).to_dict(orient='list')
d_test = X_test.to_dict(orient="list")

feature_names = list(d_demo.keys())
label_name = feature_names[-1]
del d_demo[0]
del d_test[0]
discrete_attributes = [1, 2, 5, 6, 7]

classifier = ID3()
classifier.train(d_demo, label_name, discrete_attributes)
my_pred = classifier.test(d_test, discrete_attributes)
accuracy_score(y_test, my_pred)

0.6576576576576577

## 50 runs (10 * 5 folds)

In [528]:
df[8].unique()

array(['cp', 'im', 'imS', 'imL', 'imU', 'om', 'omL', 'pp'], dtype=object)

In [71]:
from sklearn.metrics import accuracy_score 
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold

df = pd.read_csv('./ecoli.data', header=None)
target = df[8]
skf = StratifiedKFold(n_splits=5, shuffle=True)
accs = []
for run in range(10):
    print("================Run {}================".format(run))
    fold_no = 1
    for train_index, test_index in skf.split(df, target):
        train = df.loc[train_index,:]
        test = df.loc[test_index,:]
        
        X = train.drop(columns=8)
        y = train[8]
        d_demo = pd.concat([X, y], axis=1).to_dict(orient='list')
        
        X_test = test.drop(columns=8)
        y_test = test[8]
        d_test = X_test.to_dict(orient="list")
        
        feature_names = list(d_demo.keys())
        label_name = feature_names[-1]
        del d_demo[0]
        del d_test[0]
        discrete_attributes = [1, 2, 5, 6, 7]

        classifier = ID3()
        classifier.train(d_demo, label_name, discrete_attributes)
        my_pred = classifier.test(d_test, discrete_attributes)

        print("fold: ", fold_no, "===>", "accuracy: ", accuracy_score(y_test, my_pred))
        accs.append(accuracy_score(y_test, my_pred))
        fold_no += 1

================Run 0================
fold:  1 ===> accuracy:  0.6323529411764706
fold:  2 ===> accuracy:  0.5970149253731343
fold:  3 ===> accuracy:  0.7014925373134329
fold:  4 ===> accuracy:  0.6716417910447762
fold:  5 ===> accuracy:  0.6268656716417911
================Run 1================
fold:  1 ===> accuracy:  0.6617647058823529
fold:  2 ===> accuracy:  0.582089552238806


/home/bahman/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_split.py:666: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"
/home/bahman/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_split.py:666: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"


fold:  3 ===> accuracy:  0.6716417910447762
fold:  4 ===> accuracy:  0.6865671641791045
fold:  5 ===> accuracy:  0.7164179104477612
================Run 2================
fold:  1 ===> accuracy:  0.6323529411764706
fold:  2 ===> accuracy:  0.6865671641791045
fold:  3 ===> accuracy:  0.6268656716417911
fold:  4 ===> accuracy:  0.6119402985074627
fold:  5 ===> accuracy:  0.6417910447761194
================Run 3================


/home/bahman/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_split.py:666: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"
/home/bahman/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_split.py:666: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"


fold:  1 ===> accuracy:  0.7058823529411765
fold:  2 ===> accuracy:  0.5074626865671642
fold:  3 ===> accuracy:  0.6716417910447762
fold:  4 ===> accuracy:  0.6716417910447762
fold:  5 ===> accuracy:  0.6119402985074627
================Run 4================
fold:  1 ===> accuracy:  0.6764705882352942
fold:  2 ===> accuracy:  0.6268656716417911
fold:  3 ===> accuracy:  0.7014925373134329


...

fold:  4 ===> accuracy:  0.5671641791044776
fold:  5 ===> accuracy:  0.746268656716418
================Run 5================
fold:  1 ===> accuracy:  0.6617647058823529
fold:  2 ===> accuracy:  0.7164179104477612
fold:  3 ===> accuracy:  0.5970149253731343
fold:  4 ===> accuracy:  0.5522388059701493
fold:  5 ===> accuracy:  0.6716417910447762
================Run 6================
fold:  1 ===> accuracy:  0.6323529411764706


/home/bahman/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_split.py:666: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"
/home/bahman/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_split.py:666: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"


fold:  2 ===> accuracy:  0.6865671641791045
fold:  3 ===> accuracy:  0.6865671641791045
fold:  4 ===> accuracy:  0.6865671641791045
fold:  5 ===> accuracy:  0.582089552238806
================Run 7================
fold:  1 ===> accuracy:  0.6617647058823529
fold:  2 ===> accuracy:  0.6865671641791045
fold:  3 ===> accuracy:  0.6567164179104478
fold:  4 ===> accuracy:  0.5970149253731343
fold:  5 ===> accuracy:  0.6567164179104478
================Run 8================
fold:  1 ===> accuracy:  0.6029411764705882
fold:  2 ===> accuracy:  0.6268656716417911
fold:  3 ===> accuracy:  0.5970149253731343
fold:  4 ===> accuracy:  0.7313432835820896
fold:  5 ===> accuracy:  0.6268656716417911
================Run 9================
fold:  1 ===> accuracy:  0.6617647058823529


/home/bahman/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_split.py:666: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"
/home/bahman/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_split.py:666: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"


fold:  2 ===> accuracy:  0.6865671641791045
fold:  3 ===> accuracy:  0.582089552238806
fold:  4 ===> accuracy:  0.7761194029850746
fold:  5 ===> accuracy:  0.582089552238806


In [72]:
len(accs)

50

In [73]:
import statistics
print("Standard deviation: ", statistics.stdev(accs) )
print("Mean: ", statistics.mean(accs) )

Standard deviation:  0.052917967626446995
Mean:  0.6487971905179982


# letter-recognition dataset

## One Run

In [74]:
df = pd.read_csv('./letter-recognition.data', header=None)
# organize data into input and output
X = df.drop(columns=0)
y = df[0]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
d_demo = pd.concat([X_train, y_train], axis=1).to_dict(orient='list')
d_test = X_test.to_dict(orient="list")

feature_names = list(d_demo.keys())
label_name = feature_names[-1]
discrete_attributes = []

classifier = ID3()
classifier.train(d_demo, label_name, discrete_attributes)
my_pred = classifier.test(d_test, discrete_attributes)
accuracy_score(y_test, my_pred)

0.7133333333333334

## 50 runs (10 * 5 folds)

In [75]:
from sklearn.metrics import accuracy_score 
from sklearn.model_selection import StratifiedKFold

df = pd.read_csv('./letter-recognition.data', header=None)
target = df[0]
skf = StratifiedKFold(n_splits=5, shuffle=True)
accs = []
for run in range(10):
    print("================Run {}================".format(run))
    fold_no = 1
    for train_index, test_index in skf.split(df, target):
        train = df.loc[train_index,:]
        test = df.loc[test_index,:]

        X = train.drop(columns=0)
        y = train[0]
        d_demo = pd.concat([X, y], axis=1).to_dict(orient='list')
        
        X_test = test.drop(columns=0)
        y_test = test[0]
        d_test = X_test.to_dict(orient="list")

        feature_names = list(d_demo.keys())
        label_name = feature_names[-1]
        discrete_attributes = []
        
        classifier = ID3()
        classifier.train(d_demo, label_name, discrete_attributes)
        my_pred = classifier.test(d_test, discrete_attributes)

        print("fold: ", fold_no, "===>", "accuracy: ", accuracy_score(y_test, my_pred))
        accs.append(accuracy_score(y_test, my_pred))
        fold_no += 1

================Run 0================
fold:  1 ===> accuracy:  0.72625
fold:  2 ===> accuracy:  0.7285
fold:  3 ===> accuracy:  0.72525
fold:  4 ===> accuracy:  0.72675
fold:  5 ===> accuracy:  0.72675
================Run 1================
fold:  1 ===> accuracy:  0.731
fold:  2 ===> accuracy:  0.7215
fold:  3 ===> accuracy:  0.739
fold:  4 ===> accuracy:  0.73175
fold:  5 ===> accuracy:  0.71875
================Run 2================
fold:  1 ===> accuracy:  0.7275
fold:  2 ===> accuracy:  0.7335
fold:  3 ===> accuracy:  0.72875
fold:  4 ===> accuracy:  0.724
fold:  5 ===> accuracy:  0.734
================Run 3================
fold:  1 ===> accuracy:  0.71875
fold:  2 ===> accuracy:  0.73175
fold:  3 ===> accuracy:  0.73275
fold:  4 ===> accuracy:  0.733
fold:  5 ===> accuracy:  0.729
================Run 4================
fold:  1 ===> accuracy:  0.7285
fold:  2 ===> accuracy:  0.732
fold:  3 ===> accuracy:  0.72825
fold:  4 ===> accuracy:  0.72225
fold:  5 ===> accuracy:  0.7365
=====

In [534]:
len(accs)

50

In [535]:
import statistics
print("Standard deviation: ", statistics.stdev(accs) )
print("Mean: ", statistics.mean(accs) )

Standard deviation:  0.0065344456223796685
Mean:  0.7282
